In [1]:
# Data manipulation
import pandas as pd
import numpy as np

In [2]:
# Pre-processing
import glob, re, os, sys, random
import datetime

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, strip_multiple_whitespaces, strip_non_alphanum, stem_text, remove_stopwords

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

## Data Pre-processing

In [3]:
df = pd.read_json(r"../../data/processed/df_eng_clean_filtered_2023_03_11.json")

In [4]:
df['count'] = df['sec_text'].str.split().apply(len)

Remove non-alphanum, digits, punctuations
(Don't lowercase and remove stopwords yet)

In [5]:
preprocess_function = [lambda x: x, strip_tags, strip_punctuation, strip_non_alphanum, strip_multiple_whitespaces, strip_numeric] #x.lower() remove_stopwords, stem_text

df['text_clean'] = df['sec_text'].apply(lambda x: " ".join(preprocess_string(str(x), preprocess_function)))

Lemmatize text, no stemming

In [6]:
lemmatizer = WordNetLemmatizer()
# stemmer = PorterStemmer()

def stem_lemmatize(text):
    #stemmed = [stemmer.stem(token) for token in word_tokenize(text)]
    lemmatized = [lemmatizer.lemmatize(token) for token in word_tokenize(text)]
    processed_text = ' '.join(lemmatized)
    return processed_text

In [7]:
df['text_clean'] = [stem_lemmatize(text) for text in df['text_clean']]

Get length of text_clean

In [8]:
df['count_text'] = df['text_clean'].str.split().apply(len)

In [9]:
print("min:", min(df['count_text']), 
    "\nmax:", max(df['count_text']))

min: 13 
max: 233652


In [10]:
df[df['count_text']<=20].article_new.value_counts()

simplified    112
Name: article_new, dtype: int64

In [11]:
# no of cases by article_new
df.groupby('article_new')['case_num'].nunique().sort_index()

article_new
article6(1)(b)    1284
article6(2)        201
article8(1)         32
article8(2)         57
article8(3)          9
referral            59
simplified        3194
Name: case_num, dtype: int64

Remove dups (again)

In [12]:
# df = pd.read_json(r"../../data/processed/pre-processed_2023_03_11.json")

df.drop_duplicates(subset=['article_new', 'case_num', 'section_fin'], keep='first', inplace=True)
df=df.reset_index(drop=True)

Remove common words

In [13]:
# Define a regular expression pattern to match the unwanted words
remove_words = ['statement objection', 'en', 'phase ii', 'rfi', 'request information',
                'european commission', 'commission européenne bruxelles', 'europese commissie brussel', 'belgium telephone',
                'confidential information disclosed', 'bruxelles europese', 'européenne', 'commissie', 'europese', 'tele', 'brussel', 'bruxelles',
                'belgique', 'tel', 'belgïë',
                'enclosed square bracket', 'marked asterisk']
remove_pattern = re.compile(r'\b(' + '|'.join(remove_words) + r')\b', flags=re.IGNORECASE)
df['text_clean'] = df['text_clean'].replace(remove_pattern, '', regex=True)

Relabel wrong labels via merger_info

In [14]:
# df = pd.read_json(r"../../data/processed/pre-processed_2023_03_20.json")
df.loc[df['case_num'].isin(['M.4844', 'M.5020', 'M.8124']), 'article_new'] = 'article6(2)'

Add labels

In [15]:
# Phase 1 (0) vs. Phase 2 (1)
df['phase2'] = np.where((df['article_new'].isin(['article6(1)(b)', 'article6(2)'])), 0, 1)
df['phase2'] = np.where((df['article_new'].isin(['referral', 'simplified'])), df['article_new'], df['phase2'])

df.groupby('phase2')['case_num'].nunique()

phase2
0             1485
1               98
referral        58
simplified    3194
Name: case_num, dtype: int64

In [16]:
df['phase2'].value_counts()

0             5070
simplified    3194
1              292
referral       150
Name: phase2, dtype: int64

In [17]:
# df['wc'] =""
# df['wc'] = np.where((df['article_new'].isin(['article6(2)', 'article8(2)'])), 1, 0)
# df.groupby('wc')['case_num'].nunique()
# df[(df['wc']==1)]['article_new'].value_counts()
# df[(df['wc']==1)].groupby('article_new')['case_num'].nunique() # is there a duplicate by casse_num but different filename?

In [18]:
# With conditions (1) vs. Without conditions (0)
df['wc'] = np.where((df['article_new'].isin(['article6(2)', 'article8(2)'])), 1, 0)
df['wc'] = np.where((df['article_new'].isin(['referral', 'simplified', 'article8(3)'])), df['article_new'], df['wc'])

df.groupby('wc')['case_num'].nunique()


wc
0              1313
1               260
article8(3)       9
referral         58
simplified     3194
Name: case_num, dtype: int64

In [19]:
df['wc'].value_counts()

0              4492
simplified     3194
1               842
referral        150
article8(3)      28
Name: wc, dtype: int64

In [20]:
# Potentially anticompetitive vs. No harm to competition
df['competition'] = np.where((df['article_new'].isin(['article6(2)', 'article8(2)', 'article8(3)'])), 1, 0)
df['competition'] = np.where((df['article_new'].isin(['referral', 'simplified'])), df['article_new'], df['competition'])

df.groupby('competition')['case_num'].nunique()

competition
0             1313
1              269
referral        58
simplified    3194
Name: case_num, dtype: int64

In [21]:
df['competition'].value_counts()

0             4492
simplified    3194
1              870
referral       150
Name: competition, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8706 entries, 0 to 8705
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   year         8706 non-null   int64 
 1   article_new  8706 non-null   object
 2   case_num     8706 non-null   object
 3   file         8706 non-null   object
 4   section_fin  8706 non-null   object
 5   len_pdf      8706 non-null   int64 
 6   sec_text     8706 non-null   object
 7   count        8706 non-null   int64 
 8   text_clean   8706 non-null   object
 9   count_text   8706 non-null   int64 
 10  phase2       8706 non-null   object
 11  wc           8706 non-null   object
 12  competition  8706 non-null   object
dtypes: int64(4), object(9)
memory usage: 884.3+ KB


In [23]:
# save json file name
date = datetime.date.today().strftime('%Y_%m_%d')

file_name = f"../../data/processed/pre-processed_{date}.json"
if os.path.exists(file_name):
    os.remove(file_name)

# save file as json
df.to_json(file_name)

Load merger_info with NACE code data

In [159]:
directory = f"../../data/decision_types/"

dfs = []

for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        info = pd.read_excel(os.path.join(directory, filename))
        info.columns = map(str.lower, info.columns)
        dfs.append(info)

merger_info = pd.concat(dfs, ignore_index=True)


In [160]:
# change to 6.2 for M.4719
merger_info.loc[merger_info['case_code'].isin(['M.4719']), 'article'] = 'article6(2)'

In [161]:
merger_info.article.value_counts()

article6(1)(b)    1598
article6(2)        242
article8(2)         73
article8(1)         36
article8(3)         13
Name: article, dtype: int64

In [162]:
merger_info.head()

,case_code,title,regulation_id,notification_date,simpl_proc,last_decision_date,deadline_date,nace_code,article
0,M.7351,HENKEL / SPOTLESS GROUP,Council Regulation 139/2004,2014-08-25,0,2014-09-25,2014-09-29,"Manufacture of soap and detergents, cleaning a...",article6(1)(b)
1,M.7395,MEXICHEM / VESTOLIT,Council Regulation 139/2004,2014-10-10,0,2014-11-13,2014-11-14,Manufacture of rubber and plastic products,article6(1)(b)
2,M.7369,SANTANDER / PSA / JVS,Council Regulation 139/2004,2014-11-05,0,2014-12-03,2014-12-10,"Other financial service activities, except ins...",article6(1)(b)
3,M.7458,IBM / INF BUSINESS OF DEUTSCHE LUFTHANSA,Council Regulation 139/2004,2014-11-18,0,2014-12-15,2014-12-23,"Computer programming, consultancy and related ...",article6(1)(b)
4,M.6242,LACTALIS / PARMALAT,Council Regulation 139/2004,2011-05-04,0,2011-06-14,2011-06-14,Manufacture of dairy products,article6(1)(b)


In [163]:
# # save json file name
# date = datetime.date.today().strftime('%Y_%m_%d')

# file_name = f"../../data/processed/merger_info_{date}.json"
# if os.path.exists(file_name):
#     os.remove(file_name)

# # save file as json
# merger_info.to_json(file_name)

In [164]:
df1=df[~df['article_new'].isin(['referral','simplified'])]

In [165]:
df1.article_new.value_counts()

article6(1)(b)    4398
article6(2)        672
article8(2)        170
article8(1)         94
article8(3)         28
Name: article_new, dtype: int64

In [166]:
merged_df = pd.merge(df1, merger_info, how='left', left_on=['case_num', 'article_new'], right_on=['case_code', 'article'])

In [167]:
merger_info['case_code'][merger_info['case_code'].duplicated()]

1905    M.7000
Name: case_code, dtype: object

In [168]:
merger_info[merger_info['case_code'] == 'M.7000']

,case_code,title,regulation_id,notification_date,simpl_proc,last_decision_date,deadline_date,nace_code,article
1675,M.7000,LIBERTY GLOBAL / ZIGGO,Council Regulation 139/2004,2018-04-04,0,2018-05-30,NaT,Information service activities; Programming an...,article6(2)
1905,M.7000,LIBERTY GLOBAL / ZIGGO,Council Regulation 139/2004,2018-04-04,0,2018-05-30,NaT,Information service activities; Programming an...,article8(2)


In [169]:
merged_df[merged_df['case_code'].isnull()]['case_num'].unique()

array([], dtype=object)

In [170]:
merged_df[merged_df['case_code'].isnull()][['case_num','article_new']]

,case_num,article_new


In [171]:
merger_info[merger_info['case_code'].isin(['M.4719'])] # changed to actually 6.2

,case_code,title,regulation_id,notification_date,simpl_proc,last_decision_date,deadline_date,nace_code,article
1089,M.4719,HEIDELBERGCEMENT / HANSON,Council Regulation 139/2004,2007-07-03,0,2007-08-07,2007-08-07,"Manufacture of cement, lime and plaster; Manuf...",article6(2)


Merge pre-processed and NACE codes

In [24]:
# df = pd.read_json(r"../../data/processed/pre-processed_2023_04_04.json")

merger_info = pd.read_json(r"../../data/processed/merger_info_2023_03_25.json")

In [25]:
df_merged = pd.merge(df, merger_info, how='left', left_on=['case_num', 'article_new'], right_on=['case_code', 'article'])

In [26]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8706 entries, 0 to 8705
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                8706 non-null   int64  
 1   article_new         8706 non-null   object 
 2   case_num            8706 non-null   object 
 3   file                8706 non-null   object 
 4   section_fin         8706 non-null   object 
 5   len_pdf             8706 non-null   int64  
 6   sec_text            8706 non-null   object 
 7   count               8706 non-null   int64  
 8   text_clean          8706 non-null   object 
 9   count_text          8706 non-null   int64  
 10  phase2              8706 non-null   object 
 11  wc                  8706 non-null   object 
 12  competition         8706 non-null   object 
 13  case_code           5362 non-null   object 
 14  title               5362 non-null   object 
 15  regulation_id       5362 non-null   object 
 16  notifi

In [27]:
# save json file name
date = datetime.date.today().strftime('%Y_%m_%d')

file_name = f"../../data/processed/pre-processed_merged_{date}.json"
if os.path.exists(file_name):
    os.remove(file_name)

# save file as json
df_merged.to_json(file_name)

Pre-process with NLTk backup

In [103]:
# nltk.download("stopwords")
# from nltk.corpus import stopwords

In [104]:
# nltk.download('punkt')

In [105]:
## Removal of stopwords, punctuations, numeric characters
# def preprocess_corpus(texts):
#     eng_stopwords = set(stopwords.words("english"))
#     def remove_stops_digits(tokens):
#         token_list =  [token.lower() for token in tokens if token not in eng_stopwords and token not in punctuation and token.isdigit() == False]
#         processed_text = ' '.join(token_list)
#         return processed_text
#     return [remove_stops_digits(word_tokenize(text)) for text in texts]

Coreference resolution

In [ ]:
# df = pd.read_json(r"../../data/processed/pre-processed_2023_03_11.json")

In [89]:
# #installing neuralcoref from source
# !git clone https://github.com/huggingface/neuralcoref.git
# !cd "D:\Desktop\Thesis\predicting-merger-decision-outcomes\src\python\notebook\neuralcoref"
# !pip install -r requirements.txt
# !pip install -e .
# !pip install spacy
# !pip install -U neuralcoref

In [95]:
# import neuralcoref

In [91]:
# import spacy

In [92]:
# nlp = spacy.load('en_core_web_lg') 

In [ ]:
# neuralcoref.add_to_pipe(nlp)

In [93]:
# import spacy
# nlp = spacy.load('en')

In [ ]:
# def coref_res(texts):
#     doc = nlp(texts)
#     clean = doc._.coref_resolved
#     return clean

# df['text_clean'] = [coref_res(text) for text in df['text_clean']]